In [1]:
import torch

In [12]:
import torch

def fill_zero_places(a):
    a = torch.cat([torch.tensor([-1], device=a.device), a])
    nonzero_indices = torch.nonzero(a).squeeze()
    diff = torch.diff(torch.cat([nonzero_indices, torch.tensor([a.shape[0]], device=a.device)]))
    b = a[nonzero_indices].repeat_interleave(diff)
    b[b < 0] = 0
    c = b[1:]
    return c

def shift_values_to_next_available_zero(a):
    b = (a > 0).int()
    b = torch.cat([torch.tensor([0], device=a.device), b])
    c = torch.arange(b.shape[0], device=a.device) * b
    c[-1] = 0
    d = c[1:]
    e = torch.roll(fill_zero_places(d), 1)
    e[0] = 0
    f = torch.zeros_like(a)
    f.scatter_add_(0, e.long(), a)
    return f

def fill_zero_values_ascending_rest_with_zero(a):
    b = (a > 0).int()
    cumsum = torch.cumsum(1 - b, dim=0)
    c = (cumsum * (1 - b) - torch.maximum(b * cumsum, torch.zeros_like(cumsum)).cummax(dim=0)[0]) * (1 - b)
    return c

def find_one_zero_pattern(a):
    b = torch.roll(a.int(), -1)
    b[-1] = 1
    c = a - b
    d = (c == 1).int()
    return d

def fill_zero_values_descending_rest_with_zero(a):
    b = fill_zero_values_ascending_rest_with_zero(a)
    c = (b > 0).int()
    d = find_one_zero_pattern(c)
    e = d * b
    f = shift_values_to_next_available_zero(e)
    g = fill_zero_places(f)
    h = ((g + 1) - b) * c
    return h

def find_values_that_must_be_reordered(a):
    b, _ = torch.cummin(a.flip(0), dim=0)
    b = b.flip(0)
    c = torch.cat([(1 - (b[1:] - b[:-1]) > 0).int(), torch.tensor([0], device=a.device)])
    d = a * c
    e = ((d >= b).int() * c).int()
    f = (1 - c)
    g = fill_zero_values_descending_rest_with_zero(f)
    h = d - g
    i = ((h > b).int() * c | (a >= b).int() * c).int()
    return i

def adjust_cars_no_lane_change(a):
    b = a[-1] - a
    m = find_values_that_must_be_reordered(b).int()
    c = torch.where((torch.roll(m, 1) == 1) & (m == 0), torch.tensor(1, device=a.device), torch.tensor(0, device=a.device))
    d = (torch.cumsum(1 - c, dim=0) * (1 - c) - torch.maximum(c * torch.cumsum(1 - c, dim=0), torch.zeros_like(c)).cummax(dim=0)[0]) * (1 - c)
    e = a * c
    nonzero_indices = torch.nonzero(e).squeeze()
    diff = torch.diff(torch.cat([nonzero_indices, torch.tensor([e.shape[0]], device=a.device)]))
    f = e[nonzero_indices].repeat_interleave(diff)
    g = torch.roll(torch.argsort(e).cummax(dim=0)[0] * c, 1)
    h = torch.zeros_like(e)
    h.scatter_(0, g.long(), e)
    i = h.cummax(dim=0)[0]
    j = i - d
    k = fill_zero_values_descending_rest_with_zero(c)
    l = i - k
    n = 1 - m
    o = fill_zero_values_descending_rest_with_zero(n) * m
    p = (i - o) * m
    q = p + n * a
    return q

In [13]:
a = torch.tensor([ 7,  4,  2,  6,  3,  5,  8,  6, 14,  4,  8, 13,  3,  2,  6,  7,  7,  17,  5,  3,  6, 23,  9, 23,  5,  6,  3, 10, 30, 35])
adjust_cars_no_lane_change(a)

RuntimeError: zero-dimensional tensor (at position 0) cannot be concatenated

In [11]:
count = 30
lane_num_tracks = torch.tensor([198, 222])
cars_position = torch.arange(0, count)
cars_speed = torch.randint(0,10, (count, 1)).flatten()

cars_position, cars_speed

(tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 tensor([1, 3, 7, 5, 0, 3, 5, 9, 6, 5, 3, 2, 1, 5, 8, 2, 2, 3, 6, 1, 0, 4, 2, 6,
         3, 1, 0, 3, 9, 1]))